Import lines

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import random

from pandas import *

# SK-learn libraries.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA


Merge the test data into a single csv.

In [31]:
path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
train_data_filenames = glob.glob(path + "/subj1_series[1-2]*data.csv") #load only subject 1, series 1-3
list_ = []
Start_Time = time.time()
print time.time(), "Merging..."

for file_ in train_data_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
train_data = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#now with event data
path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
train_event_filenames = glob.glob(path + "/subj1_series[1-2]*events.csv") #load only subject 1, series 1-3
list_ = []

Start_Time = time.time()
print time.time(), "Merging..."

for file_ in train_event_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
train_events = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#do the same thing and now get your testing data
train_data_filenames = glob.glob(path + "/subj1_series3*data.csv") #load only subject 1, series 1-3
list_ = []
Start_Time = time.time()
print time.time(), "Merging..."

for file_ in train_data_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
test_data = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#now with event data
path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
test_event_filenames = glob.glob(path + "/subj1_series3*events.csv") #load only subject 1, series 1-3
list_ = []

Start_Time = time.time()
print time.time(), "Merging..."

for file_ in test_event_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
test_events = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"

1438144612.17 Merging...
1438144612.17 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series1_data.csv
1438144613.34 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series2_data.csv
Merge Complete: 3.87000012398 total seconds
1438144616.05 Merging...
1438144616.05 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series1_events.csv
1438144616.36 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series2_events.csv
Merge Complete: 1.00999999046 total seconds
1438144617.06 Merging...
1438144617.06 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series3_data.csv
Merge Complete: 1.37999987602 total seconds
1438144618.44 Merging...
1438144618.44 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series3_events.csv
Merge Complete: 0.238000154495 total seconds


Train a simple model and make predictions on the test data.

In [32]:
Train_Labels_HandStart =  train_events['HandStart']
Train_Labels_FirstDigitTouch =  train_events['FirstDigitTouch']
Train_Labels_BothStartLoadPhase =  train_events['BothStartLoadPhase']
Train_Labels_LiftOff =  train_events['LiftOff']
Train_Labels_Replace =  train_events['Replace']
Train_Labels_BothReleased =  train_events['BothReleased']
Train_Labels_Combined = train_events.HandStart.map(str) + ',' + train_events.FirstDigitTouch.map(str) + ',' + train_events.BothStartLoadPhase.map(str) + ',' + train_events.LiftOff.map(str) + ',' + train_events.Replace.map(str) + ',' + train_events.BothReleased.map(str)


Train_Data = train_data.iloc[0:,1:] # select only column data


Test_Labels_HandStart =  test_events['HandStart']
Test_Labels_FirstDigitTouch =  test_events['FirstDigitTouch']
Test_Labels_BothStartLoadPhase =  test_events['BothStartLoadPhase']
Test_Labels_LiftOff =  test_events['LiftOff']
Test_Labels_Replace =  test_events['Replace']
Test_Labels_BothReleased =  test_events['BothReleased']
Test_Labels_Combined = test_events.HandStart.map(str) + ',' + test_events.FirstDigitTouch.map(str) + ',' + test_events.BothStartLoadPhase.map(str) + ',' + test_events.LiftOff.map(str) + ',' + test_events.Replace.map(str) + ',' + test_events.BothReleased.map(str)


Test_Data = test_data.iloc[0:,1:] # select only data columns

print pd.unique(Train_Labels_Combined.values.ravel())



['0,0,0,0,0,0' '1,0,0,0,0,0' '0,1,0,0,0,0' '0,1,1,0,0,0' '0,0,1,0,0,0'
 '0,0,0,1,0,0' '0,0,0,0,1,0' '0,0,0,0,1,1' '0,0,0,0,0,1' '0,1,1,1,0,0'
 '0,0,1,1,0,0']


Define functions for feature extraction

In [28]:
#Remove the channels we don't want 
def Remove_Channels(df):
    df.drop(df.columns[[14,15,19,20,21,24,25,26,27,28,29,30,31]], axis=1, inplace=True)
    return df

#bin the time 
def Bin_Time(num_rows,num_bins):
    Bin_Size = num_rows/num_bins
    Bins = np.zeros(shape=(num_rows,num_bins))
    Bin_Min = 0
    Bin_Max = Bin_Size
    for i in range(0,num_bins):
        Bins[Bin_Min:Bin_Max,i] = 1
        Bin_Min = Bin_Min + Bin_Size
        Bin_Max = Bin_Max + Bin_Size
    return Bins


#run PCA and return the number of PCs that explain the given amount of variance. 
def extractFeatures_PCA(df,Test_df, PercentVarExplained):
    data = df.as_matrix()
    print len(data)
    test_data = Test_df.as_matrix()
    pca = PCA()
    pca.fit(data)  
    
    Explained_Variance_Ratios = pca.explained_variance_ratio_
    for i in range(1,len(Explained_Variance_Ratios)):
        if sum(Explained_Variance_Ratios[0:i]) >= PercentVarExplained:
                   NumPCs = i + 1 #add 1 since numpy array ranges are not inclusive
                   break
    PCA_Projections = pca.transform(data)[:,0:NumPCs]
    PCA_Projections_Test = pca.transform(test_data)[:,0:NumPCs]
    return PCA_Projections,PCA_Projections_Test



#return rolling mean of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_mean(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_mean(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'mean' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling variance of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_var(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_var(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'var' + str(i)})
    return np.array(new_df.astype('float32'))


#return rolling min of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_min(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_min(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'min' + str(i)})
    return np.array(new_df.astype('float32'))


#return rolling max of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_max(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_max(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'max' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling skew of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_skew(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_skew(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'skew' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling skew of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_kurt(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_kurt(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'kurt' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling quantile of each column in a pandas dataframe with a given window and quantile. Returns df of same size. 
def df_rolling_quantile(df,window,quantile):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_quantile(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'Pct'+ quantile + "_" + str(i)})
    return np.array(new_df.astype('float32'))


#BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
#return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_corr(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(pd.rolling_corr(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)



#BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
#return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_cov(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(rolling_cov(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)




[[ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]]


In [33]:
Train_Data = Remove_Channels(Train_Data)
Test_Data = Remove_Channels(Test_Data)
print "Complete"
print Train_Data.head()

Complete
   Fp1  Fp2   F7   F3   Fz   F4   F8  FC5  FC1  FC2  FC6   T7   C3   Cz  TP9  \
0  -31  363  211  121  211   15  717  279   35  158  543 -166  192  230  128   
1  -29  342  216  123  222  200  595  329   43  166  495 -138  201  233  185   
2 -172  278  105   93  222  511  471  280   12  177  534 -163  198  207  145   
3 -272  263  -52   99  208  511  428  261   27  180  525 -310  212  221  115   
4 -265  213  -67   99  155  380  476  353   32  165  507 -320  242  230  180   

   CP5  CP1   P7   P3  
0   59  272  536  348  
1   47  269  529  327  
2   52  250  511  319  
3   41  276  521  336  
4   89  288  550  324  


In [20]:
Pct_Variance_Explained = .7
Rolling_Window = 50

PCA_Projections_Train, PCA_Projections_Test = extractFeatures_PCA(Train_Data,Test_Data,Pct_Variance_Explained)

PCA_Rolling_Means_Train = df_rolling_mean(pd.DataFrame(PCA_Projections_Train),Rolling_Window)
PCA_Rolling_Var_Train = df_rolling_var(pd.DataFrame(PCA_Projections_Train),Rolling_Window)
PCA_Rolling_Skewness_Train = df_rolling_skew(pd.DataFrame(PCA_Projections_Train),Rolling_Window)
PCA_Rolling_Kurtosis = df_rolling_kurt(pd.DataFrame(PCA_Projections_Train),Rolling_Window)
PCA_Rolling_Min_Train = df_rolling_min(pd.DataFrame(PCA_Projections_Train),Rolling_Window)
PCA_Rolling_Max_Train = df_rolling_max(pd.DataFrame(PCA_Projections_Train),Rolling_Window)


PCA_Rolling_Means_Test = df_rolling_mean(pd.DataFrame(PCA_Projections_Test),Rolling_Window)
PCA_Rolling_Var_Test = df_rolling_var(pd.DataFrame(PCA_Projections_Test),Rolling_Window)
PCA_Rolling_Skewness_Test = df_rolling_skew(pd.DataFrame(PCA_Projections_Test),Rolling_Window)
PCA_Rolling_Kurtosis_Test = df_rolling_kurt(pd.DataFrame(PCA_Projections_Test),Rolling_Window)
PCA_Rolling_Min_Test = df_rolling_min(pd.DataFrame(PCA_Projections_Test),Rolling_Window)
PCA_Rolling_Max_Test = df_rolling_max(pd.DataFrame(PCA_Projections_Test),Rolling_Window)

Feature_Array_Train = np.concatenate((PCA_Projections_Train,PCA_Rolling_Means_Train,PCA_Rolling_Var_Train,PCA_Rolling_Min_Train,PCA_Rolling_Max_Train),axis=1)
Feature_Array_Test = np.concatenate((PCA_Projections_Test,PCA_Rolling_Means_Test,PCA_Rolling_Var_Test,PCA_Rolling_Min_Test,PCA_Rolling_Max_Test),axis=1)


Train_Labels_Combined = np.array(Train_Labels_Combined)
Test_Labels_Combined = np.array(Test_Labels_Combined)


391450


In [25]:
Rolling_Window = 20

RawData_Rolling_Means_Train = df_rolling_mean(pd.DataFrame(Train_Data),Rolling_Window)
RawData_Rolling_Var_Train = df_rolling_var(pd.DataFrame(Train_Data),Rolling_Window)
RawData_Rolling_Skewness_Train = df_rolling_skew(pd.DataFrame(Train_Data),Rolling_Window)
# RawData_Rolling_Kurtosis = df_rolling_kurt(pd.DataFrame(Train_Data),Rolling_Window)
RawData_Rolling_Min_Train = df_rolling_min(pd.DataFrame(Train_Data),Rolling_Window)
RawData_Rolling_Max_Train = df_rolling_max(pd.DataFrame(Train_Data),Rolling_Window)


RawData_Rolling_Means_Test = df_rolling_mean(pd.DataFrame(Test_Data),Rolling_Window)
RawData_Rolling_Var_Test = df_rolling_var(pd.DataFrame(Test_Data),Rolling_Window)
RawData_Rolling_Skewness_Test = df_rolling_skew(pd.DataFrame(Test_Data),Rolling_Window)
# RawData_Rolling_Kurtosis_Test = df_rolling_kurt(pd.DataFrame(Test_Data),Rolling_Window)
RawData_Rolling_Min_Test = df_rolling_min(pd.DataFrame(Test_Data),Rolling_Window)
RawData_Rolling_Max_Test = df_rolling_max(pd.DataFrame(Test_Data),Rolling_Window)

In [24]:
RawData_Feature_Array_Train = np.concatenate((Train_Data,RawData_Rolling_Skewness_Train,RawData_Rolling_Min_Train,RawData_Rolling_Max_Train),axis=1)
RawData_Feature_Array_Test = np.concatenate((Test_Data,RawData_Rolling_Skewness_Test,RawData_Rolling_Min_Test,RawData_Rolling_Max_Test),axis=1)

NameError: name 'RawData_Rolling_Min_Train' is not defined

In [ ]:
Logistic_Reg = LogisticRegression(C=.01,penalty = 'l2')  

# Logistic_Reg.fit(Feature_Array_Train, Train_Labels_Combined)
# print "Combined Score =", Logistic_Reg.score(Feature_Array_Test, Test_Labels_Combined)

#try optimizing for C
C_Values = []
f1scores_Logistic_Reg = []
highest_f1_Logistic_Reg = 0
Best_C = 0
for i in range(20,300,40):
    c = np.true_divide(i,100) #range function doesn't accept float values. Do this as a workaround
    Logistic_Reg = LogisticRegression(C = c, penalty = 'l2')       
    Logistic_Reg.fit(Feature_Array_Train, Train_Labels_HandStart)
    C_Values.append(c)
    f1 = Logistic_Reg.score(Feature_Array_Test, Test_Labels_HandStart)
    f1scores_Logistic_Reg.append(f1)
    if f1 > highest_f1_Logistic_Reg:
        highest_f1_Logistic_Reg = f1
        Best_C = c


print "Logistic Regression Highest F1 =", highest_f1_Logistic_Reg
print "Optimized C = ", Best_C
plt.plot(C_Values,f1scores_Logistic_Reg)
plt.show()

Logistic_Reg.fit(Feature_Array_Train, Train_Labels_HandStart)
print "HandStart Score =", Logistic_Reg.score(Feature_Array_Test, Test_Labels_HandStart)

# Logistic_Reg.fit(df_Features_Train, Train_Labels_FirstDigitTouch)
# print "FirstDigitTouch Score =", Logistic_Reg.score(df_Features_Test, Test_Labels_FirstDigitTouch)

# Logistic_Reg.fit(df_Features_Train, Train_Labels_BothStartLoadPhase)
# print "BothStartLoadPhase Score =", Logistic_Reg.score(df_Features_Test, Test_Labels_BothStartLoadPhase)

# Logistic_Reg.fit(df_Features_Train, Train_Labels_LiftOff)
# print "LiftOff Score =", Logistic_Reg.score(df_Features_Test, Test_Labels_LiftOff)

# Logistic_Reg.fit(df_Features_Train, Train_Labels_Replace)
# print "Replace Score =", Logistic_Reg.score(df_Features_Test, Test_Labels_Replace)

# Logistic_Reg.fit(df_Features_Train, Train_Labels_BothReleased)
# print "BothReleased Score =", Logistic_Reg.score(df_Features_Test, Test_Labels_BothReleased)

In [23]:
Logistic_Reg = LogisticRegression(C = 1, penalty = 'l1',tol=.01)      
Logistic_Reg.fit(RawData_Feature_Array_Train, Train_Labels_HandStart)
print "HandStart Score =", Logistic_Reg.score(RawData_Feature_Array_Test, Test_Labels_HandStart)
Logistic_Reg.coef_

HandStart Score = 0.980685985277


array([[ -9.08241945e-04,   6.99887074e-05,  -3.52455316e-03,
         -5.20895852e-04,   3.97556049e-04,  -6.43212664e-04,
         -7.00928486e-05,  -1.43071587e-03,   8.86240452e-04,
         -2.17075224e-03,   1.79851713e-04,  -1.58706337e-04,
         -3.06558795e-03,   9.50746138e-04,  -2.52680511e-03,
          3.72770433e-04,   2.46220752e-04,  -2.81254580e-03,
          4.12971001e-04,  -7.39341416e-04,  -3.40829942e-04,
          2.90830234e-04,   7.90061400e-04,  -1.14708669e-03,
          8.87791889e-04,   1.31484697e-03,  -3.83253136e-04,
         -1.88151252e-03,   1.41882009e-03,  -3.64354197e-03,
          6.67256268e-04,   3.30141653e-03,  -2.58768003e-02,
          6.49569088e-02,  -2.62481864e-02,  -4.97985572e-02,
         -1.13873696e-03,  -6.86990858e-02,   8.28691921e-02,
         -9.78026858e-02,  -2.88553646e-02,   4.23741738e-02,
          3.49304940e-02,   6.62114710e-02,   1.24500284e-01,
          9.78206331e-02,  -8.17871661e-02,  -9.12658280e-02,
        

In [11]:
model = LogisticRegression(penalty='l2')
model.fit(train_features, labels[class_to_test])

# print auc
predicted = [model.predict(test_feature)[0] for test_feature in test_features]
predicted = np.array(predicted)
fpr, tpr, thresholds = metrics.roc_curve(t_labels[class_to_test], predicted, pos_label=1)
print(metrics.auc(fpr, tpr))

array([[  7.94138575e-04,   2.17986863e-03,  -5.80203517e-03,
         -2.12478661e-04,  -7.51109627e-05,   2.63227380e-04,
          6.70923003e-04,  -1.35098331e-03,   3.08980826e-03,
         -4.21881991e-03,   8.23844584e-04,   1.49916112e-03,
         -1.68620524e-03,   1.98260935e-04,  -3.03162043e-03,
         -9.56549728e-04,   2.45055473e-04,  -3.68700077e-03,
          9.13275596e-04,   3.01850623e-04,  -4.13787212e-04,
          3.08575558e-05,   2.07674908e-03,  -9.20516404e-04,
          1.86429017e-03,   2.32790012e-03,  -1.46421479e-03,
         -3.35731230e-03,   1.34172566e-03,  -4.89636966e-03,
          2.44759584e-03,   9.48581760e-03,   1.04768621e-05,
         -1.93478653e-05,   6.90656137e-06,  -1.56814275e-05,
         -1.51406996e-04,   1.50997345e-05,  -1.59292434e-05,
         -5.59498976e-06,   2.04969567e-04,  -1.23037789e-04,
         -6.66066825e-06,   2.34130933e-07,  -2.31117120e-05,
          4.07752634e-04,  -1.32834314e-04,  -2.83879230e-07,
        

In [14]:
Logistic_Reg = LogisticRegression(C = 100, penalty = 'l1',tol=.01)      
Logistic_Reg.fit(Train_Data, Train_Labels_HandStart)
print "HandStart Score =", Logistic_Reg.score(Test_Data, Test_Labels_HandStart)
Logistic_Reg.coef_

HandStart Score = 0.980663008814


array([[  2.56300716e-04,   6.15128662e-04,  -2.99262791e-03,
         -1.35495359e-04,   1.42485469e-03,  -4.38628395e-04,
          8.20475761e-04,  -9.37546657e-04,   6.52229610e-04,
         -1.28483640e-03,   1.02231844e-03,  -3.73184452e-06,
         -3.05782606e-03,   1.77223605e-03,  -3.58648458e-03,
         -2.71684348e-04,   1.71934429e-04,  -1.98686683e-03,
          5.46144938e-04,  -7.60147392e-04,   4.83713221e-04,
          2.05109558e-04,   8.91211210e-04,   6.72015859e-05,
          1.30949706e-03,   1.68306197e-03,   2.39548849e-05,
         -9.67449531e-04,   1.57336310e-03,  -3.51123277e-03,
          1.24130092e-03,   4.08118280e-03]])